### Parameters

We can apply some parameters to execute the notebook for the other tables (products, customers), to do that, we can add some functions with dbutils, or we can do a pipeline to execute the notebook.

In [0]:
dbutils.widgets.text('file_name', '')

In [0]:
p_file_name = dbutils.widgets.get('file_name')

### Data Reading

In [0]:
#Check the data in the source container

#df = spark.read.format('parquet).load(f'abfss://source@databrickseterjed.dfs.core.windows.net/{p_file_name})
#df.display()

In [0]:
#Read the data in source container consider autoloader as a streaming source
df = spark.readStream.format('cloudFiles')\
        .option('cloudFiles.format', 'parquet')\
        .option('cloudFiles.schemaLocation', f'abfss://bronze@databrickseterjed.dfs.core.windows.net/checkpoint_{p_file_name}')\
        .load(f'abfss://source@databrickseterjed.dfs.core.windows.net/{p_file_name}')


### Data Writing

In [0]:
#Write the data in bronze container executing once, and consider bronze as a batch source 
df.writeStream.format('delta')\
        .outputMode('append')\
        .option('checkpointLocation', f'abfss://bronze@databrickseterjed.dfs.core.windows.net/checkpoint_{p_file_name}')\
        .option('path', f'abfss://bronze@databrickseterjed.dfs.core.windows.net/{p_file_name}')\
        .trigger(once=True)\
        .start()